In [1]:
import os
import re
import sys
import time

import numpy as np
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
import multiprocessing as mp

import requests
from getpass import getpass

import tendrils
from tendrils import api, utils

tendrils.utils.copy_from_other_config('config.ini')

For more info about the ATLAS API, go to: https://fallingstar-data.com/forcedphot/apiguide/
This code is adapted from their code.

In [2]:
def get_token(user, password):
    BASEURL = "https://fallingstar-data.com/forcedphot"
    
    if os.environ.get('ATLASFORCED_SECRET_KEY'):
        token = os.environ.get('ATLASFORCED_SECRET_KEY')
        print('Using stored token')
    else:
        data = {'username': user, 'password': password}
        resp = requests.post(url=f"{BASEURL}/api-token-auth/", data=data)

        if resp.status_code == 200:
            token = resp.json()['token']
            print(f'Your token is {token}')
            print('Store this by running/adding to your .zshrc file:')
            print(f'export ATLASFORCED_SECRET_KEY="{token}"')
        else:
            print(f'ERROR {resp.status_code}')
            print(resp.text)
            sys.exit()
            
    return token

def get_headers(token):
    
    headers = {'Authorization': f'Token {token}', 
               'Accept': 'application/json'}
    
    return headers

def submit_task(ra, dec, headers):
    BASEURL = "https://fallingstar-data.com/forcedphot"
    
    task_url = None
    while not task_url:
        with requests.Session() as s:
            # alternative to token auth
            # s.auth = ('USERNAME', 'PASSWORD')
            resp = s.post(f"{BASEURL}/queue/", headers=headers, data={
                'ra': 239.6715, 'dec': 11.24024, 'send_email': False})

            if resp.status_code == 201:  # successfully queued
                task_url = resp.json()['url']
                print(f'The task URL is {task_url}')
            elif resp.status_code == 429:  # throttled
                message = resp.json()["detail"]
                print(f'{resp.status_code} {message}')
                t_sec = re.findall(r'available in (\d+) seconds', message)
                t_min = re.findall(r'available in (\d+) minutes', message)
                if t_sec:
                    waittime = int(t_sec[0])
                elif t_min:
                    waittime = int(t_min[0]) * 60
                else:
                    waittime = 10
                print(f'Waiting {waittime} seconds')
                time.sleep(waittime)
            else:
                print(f'ERROR {resp.status_code}')
                print(resp.text)
                sys.exit()
                
    return task_url

def get_url(task_url, headers):
    
    result_url = None
    taskstarted_printed = False
    while not result_url:
        with requests.Session() as s:
            resp = s.get(task_url, headers=headers)

            if resp.status_code == 200:  # HTTP OK
                if resp.json()['finishtimestamp']:
                    result_url = resp.json()['result_url']
                    print(f"Task is complete with results available at {result_url}")
                elif resp.json()['starttimestamp']:
                    if not taskstarted_printed:
                        print(f"Task is running (started at {resp.json()['starttimestamp']})")
                        taskstarted_printed = True
                    time.sleep(2)
                else:
                    # print(f"Waiting for job to start (queued at {resp.json()['timestamp']})")
                    time.sleep(4)
            else:
                print(f'ERROR {resp.status_code}')
                print(resp.text)
                sys.exit()
                
    return result_url

def get_lightcurves(ra, dec, user, password):
    
    token = get_token(user, password)
    headers = get_headers(token)
    task_url = submit_task(ra, dec, headers)
    result_url = get_url(task_url, headers)
    
    with requests.Session() as s:
        textdata = s.get(result_url, headers=headers).text

    lc_df = pd.read_csv(StringIO(textdata.replace("###", "")), delim_whitespace=True)
    
    return lc_df

def pool_download(name, ra, dec, user, password):
    
    lc_df = get_lightcurves(ra, dec, user, password)
    lc_file = os.path.join('data', f'{name}.csv')
    lc_df.to_csv(lc_file, index=False)

In [3]:
targets_list = api.get_targets()
targets_df = pd.DataFrame(targets_list)

flows_df = targets_df[targets_df.project=='flows']
flows_df = flows_df[flows_df.target_status=='target']

In [4]:
user = 't.e.muller-bravo'
password = getpass('Password:')

Password: ··········


In [5]:
#for index, row in flows_df.iterrows():
#    name = row.target_name
#    ra, dec = row.ra, row.decl
#    
#    lc_df = get_lightcurves(ra, dec, user, password)
#    lc_file = os.path.join('data', f'{name}.csv')
#    lc_df.to_csv(lc_file, index=False)

Parallel download

In [ ]:
N = len(flows_df)
input_args = [flows_df.target_name.values, 
              flows_df.ra.values, 
              flows_df.decl.values,
              [user]*N,
              [password]*N,
             ]
input_args = list(map(list, zip(*input_args)))

pool = mp.Pool(12)
pool.starmap(pool_download, (args for args in input_args))

Your token is 6545448b543e51217af852e5878fdf4b13d58aecYour token is 6545448b543e51217af852e5878fdf4b13d58aecYour token is 6545448b543e51217af852e5878fdf4b13d58aecYour token is 6545448b543e51217af852e5878fdf4b13d58aecYour token is 6545448b543e51217af852e5878fdf4b13d58aec




Store this by running/adding to your .zshrc file:Store this by running/adding to your .zshrc file:Store this by running/adding to your .zshrc file:
Store this by running/adding to your .zshrc file:Store this by running/adding to your .zshrc file:

export ATLASFORCED_SECRET_KEY="6545448b543e51217af852e5878fdf4b13d58aec"

Your token is 6545448b543e51217af852e5878fdf4b13d58aecexport ATLASFORCED_SECRET_KEY="6545448b543e51217af852e5878fdf4b13d58aec"export ATLASFORCED_SECRET_KEY="6545448b543e51217af852e5878fdf4b13d58aec"
export ATLASFORCED_SECRET_KEY="6545448b543e51217af852e5878fdf4b13d58aec"Your token is 6545448b543e51217af852e5878fdf4b13d58aecexport ATLASFORCED_SECRET_KEY="6545448b543e51217af852e5878fdf4b13d58aec"





